<a href="https://colab.research.google.com/github/MAmirMR/Tugas_Akhir_S1/blob/main/%5BKode_1%5D_Konversi_Data_Raw_menjadi_CVS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tentang Kode:


## Tentang Umum

Kode Block ini disusun untuk menyempurnakan kode block sebelumnya yang tersusun kompleks (Kurang Intuitif). Pada bagian ini akan dipisah kode sesuai dengan fungsinya:

1.   Konversi Data Raw menjadi CVS   
**Input**: Gambar RGBs (Data Train dan Test)   
**Process 1**: Ektrkasi Color Space Gambar  
**Output**: Dataframe dikemas dalam CSV

2.    Membangun dan Mengevaluasi Model   
**Input**: Dataframe dikemas dalam CSV    
**Process 1**: Model Training  
**Process 2**: SHAP - Feature Selection   
**Process 3**: Evaluasi Model   
**Output 1**: Model Training format ONNX   
**Output 2**: Model Evalusai SHAP Method  
**Output 3**: Model Metric Value     



## Tentang Blok Kode ini

Kode **[Kode-1] Konversi Data Raw menjadi CVS.ipynb** merupakan kode yang dijalankan pertama kali. Tujuannya adalah mengubah data mentah (Berupa Gambar RGBs) menjadi data terstruktur yang dikemas dalam bentuk Dataframe berformat CSV


**INPUT**   
Gambar dengan *Color Space* **RGB** dikemas dalam bentuk folder yang sudah dikompres dengan format **ZIP**. Terdapat 2 data yang di-input yaitu **Data_Train.zip**  dan **Data_Test.zip**  

***Sususnan Folder***   
<pre>Data_[Train/Test]/
├── 0/                                  (Jam ke-0)
│  ├── 0.jpg                            (Gambar ke-0)
│  ├── 1.jpg
│  ├── 2.jpg
│  └── ...                              (Gambar ke-x)
├── 1/
│  ├── 0.jpg
│  ├── 1.jpg
│  ├── 2.jpg
│  └── ...
├── 2/
│  ├── 0.jpg
│  ├── 1.jpg
│  ├── 2.jpg
│  └── ...
└── ...                                  (Jam ke-x) </pre>


**PROCESS**
1. Import Library dan Inisiasi Dependensi
2. Load data melalui **ID URL** Google Drive
3. Ektrak file **ZIP**
4. Membaca setiap gambar dari setiap folder menggunakan *For Loop*
5. Augmentasi setiap gambar yang dibaca dengan metode **[HorizontalFlip, VerticalFlip, Rotate, ShiftScaleRotate, RandomResizedCrop]**
6. Ektraksi warna gambar dengan menghitung ***Mean*** dari area gambar
7. Konversi warna dari RGBs ke RGB Linear (Berikutnya disebut RGB)
8. Konversi RGB ke XYZ
9. Konversi XYZ ke **[HSV, YUV, Lab, CMYK]**
10. Data *Color Space* dikemas dalam bentuk **DataFrame** dengan struktur berikut:

    | Komponen | Keterangan |
    |----------|------------|
    | Feature  | R, G, B, H, S, V, Y, U, V, L, a, b, C, M, Y, K |
    | Target   | Kesegaran, Sisa Usia (Jam) |

    **Keterangan:**  
    Status kesegaran ditentukan berdasarkan nilai **TVBN** *(mg N/100 g)* sebagai berikut:

    | Status Kesegaran | Keterangan (mg N/100 g) |
    |------------------|-------------------------|
    | Segar            | TVBN < 20               |
    | Kurang Segar     | 20 ≤ TVBN ≤ 30          |
    | Tidak Segar      | TVBN > 30               |

    Dengan:   
    Catatan (Bisa berubah):   
    | Status Kesegaran | Usia Daging (jam) |
    |------------------|-------------------|
    | Segar            | 0 - 20            |
    | Kurang Segar     | 20 - 36           |
    | Tidak Segar      | 36 - 72           |    

11. Mengunduh dataset [**Dataset_Raw_Train.csv** dan **Dataset_Raw_Test.csv**]

**Output**   
File **Dataset_Raw_Train.csv** dan **Dataset_Raw_Test.csv** dengan *Feature* berupa [R, G, B, H, S, V, Y, U, V, L, a, b, C, M, Y, K] dan *Target* berupa [Kesegaran, Sisa Usia (Jam)] yang secara otomatis akan terunduh ke perangkat lokal.

# Kode Utama

## Setup Colab & Dependensi

Install Library

In [5]:
# Install Library untuk proses Augmentasi dan Ektraksi Warna
!pip install numpy==2.0.2 pandas==2.2.2 gdown==5.2.1 albumentations==2.0.8 opencv-python==4.13.0.92

Import Library

In [6]:
import os
import zipfile
import random
import numpy as np
import pandas as pd
import gdown

import cv2
import albumentations as A

from google.colab import files

Inisiasi Seed

In [7]:
# Cek fungsi random
def set_seed(seed: int):
  """
  Mengatur random seed untuk modul `random` dan `numpy`
  guna memastikan reproducibility, lalu menampilkan
  hasil uji konsistensi sebanyak 3 kali.

  Parameter
  ----------
  seed : int
      Nilai seed yang digunakan untuk inisialisasi
      generator angka acak pada modul `random`
      dan `numpy`.

  Returns
  -------
  None
  """

  n = 1
  for _ in range(3):
    SEED = seed
    random.seed(SEED)
    np.random.seed(SEED)

    # Output
    print(f"Loop Tes ke-{n}")
    print(f"Tes Random Seed: {random.random()}")
    print(f"Tes Numpy Seed: {np.random.random()} \n")
    n += 1

In [8]:
# Pemanggilan fungsi set_seed()
set_seed(42)

Loop Tes ke-1
Tes Random Seed: 0.6394267984578837
Tes Numpy Seed: 0.3745401188473625 

Loop Tes ke-2
Tes Random Seed: 0.6394267984578837
Tes Numpy Seed: 0.3745401188473625 

Loop Tes ke-3
Tes Random Seed: 0.6394267984578837
Tes Numpy Seed: 0.3745401188473625 



## Data Loading

Konfigurasi Awal

In [9]:
# Konfigurasi File ID Google Drive
TRAIN_ZIP_FILE_ID = "1ZVGaH2H-EEeNXrM03DNJtoE0PIT2pVf2"
TEST_ZIP_FILE_ID  = "1GHNpXm7ylhicvRSqKVQVrbnxby05F2mI"

# Konfigurasi Nama file hasil download
TRAIN_ZIP_NAME = "Data_Train.zip"
TEST_ZIP_NAME  = "Data_Test.zip"

Fungsi Manual *Label Encoding*

In [10]:
def freshness_label_from_hour(hour: int) -> str:
    """
    Mengonversi nilai sisa usia dalam jam menjadi label tingkat kesegaran.

    Aturan pengelompokan:
    - hour < 5            -> "Segar"
    - 8 <= hour <= 10     -> "Kurang Segar"
    - Selain itu          -> "Tidak Segar"

    Parameter
    ----------
    hour : int
        Nilai sisa usia dalam jam.

    Returns
    -------
    str
        Label kesegaran dalam bentuk string:
        "Segar", "Kurang Segar", atau "Tidak Segar".
    """

    if hour < 5:
        return "Segar"
    elif 8 <= hour <= 10:
        return "Kurang Segar"
    else:
        return "Tidak Segar"

Fungsi *Loading Data* dari Google Drive

In [11]:
def download_from_drive(file_id: str, out_path: str):
    """
    Mengunduh file dari Google Drive menggunakan file ID
    dan menyimpannya ke lokasi yang ditentukan.

    Parameter
    ----------
    file_id : str
        ID Google Drive dari file yang akan diunduh
        (bagian setelah 'id=' pada URL).
    out_path : str
        Path lengkap lokasi penyimpanan file,
        termasuk nama file dan ekstensi.

    Returns
    -------
    None
    """

    url = f"https://drive.google.com/uc?id={file_id}"
    print(f"\nDownload: {out_path}")
    gdown.download(url, out_path, quiet=False)

In [12]:
# Folder kerja
WORKDIR = "/content/raw_data"
os.makedirs(WORKDIR, exist_ok=True)

train_zip_path = os.path.join(WORKDIR, TRAIN_ZIP_NAME)
test_zip_path  = os.path.join(WORKDIR, TEST_ZIP_NAME)

# Download
download_from_drive(TRAIN_ZIP_FILE_ID, train_zip_path)
download_from_drive(TEST_ZIP_FILE_ID,  test_zip_path)


Download: /content/raw_data/Data_Train.zip


Downloading...
From: https://drive.google.com/uc?id=1ZVGaH2H-EEeNXrM03DNJtoE0PIT2pVf2
To: /content/raw_data/Data_Train.zip
100%|██████████| 17.0M/17.0M [00:00<00:00, 106MB/s] 



Download: /content/raw_data/Data_Test.zip


Downloading...
From: https://drive.google.com/uc?id=1GHNpXm7ylhicvRSqKVQVrbnxby05F2mI
To: /content/raw_data/Data_Test.zip
100%|██████████| 17.0M/17.0M [00:00<00:00, 50.1MB/s]


Fungsi Unzip File Dataset

In [13]:
def unzip_to(zip_path: str, out_dir: str):
    """
    Mengekstrak file ZIP ke direktori tujuan.

    Parameter
    ----------
    zip_path : str
        Path lengkap file ZIP yang akan diekstrak.
    out_dir : str
        Direktori tujuan tempat file ZIP akan diekstrak.
        Jika direktori belum ada, akan dibuat otomatis.

    Returns
    -------
    None
    """

    os.makedirs(out_dir, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(out_dir)

Ektrak file ZIP

In [14]:
# Lokasi Folder tujuan
EXTRACT_DIR = "/content/extracted"
TRAIN_DIR = os.path.join(EXTRACT_DIR, "Data_Train")
TEST_DIR  = os.path.join(EXTRACT_DIR, "Data_Test")

os.makedirs(EXTRACT_DIR, exist_ok=True)

# Ekstrak
unzip_to(train_zip_path, TRAIN_DIR)
unzip_to(test_zip_path,  TEST_DIR)

## Pemrosesan Data Citra

### Mendefinisikan Fungsi dan Proses

#### Proses Ektraksi Warna

Konversi sRGB ke CIE Lab Ternomalisasi

In [15]:
def rgb_to_lab_cie_normalized(rgb_uint8: np.ndarray) -> np.ndarray:
    """
    Mengonversi satu piksel RGB (uint8) ke ruang warna CIE LAB
    menggunakan OpenCV, lalu menormalkan hasilnya ke rentang 0–1.

    Proses konversi:
    1) RGB (uint8, 0–255)
    2) Konversi ke LAB menggunakan OpenCV (cv2.COLOR_RGB2LAB)
    3) Penyesuaian ke standar CIE:
       - L* diskalakan ke 0–100
       - a* dan b* digeser dengan -128
    4) Normalisasi:
       - L* → dibagi 100
       - a*, b* → diskalakan ke 0–1

    Parameter
    ----------
    rgb_uint8 : np.ndarray
        Array RGB berukuran (3,) dengan tipe uint8
        dalam rentang 0–255.

    Returns
    -------
    np.ndarray
        Array LAB ter-normalisasi berukuran (3,)
        dengan rentang nilai:
        - L_norm ∈ [0, 1]
        - a_norm ∈ [0, 1]
        - b_norm ∈ [0, 1]
    """

    rgb_uint8 = np.asarray(rgb_uint8).reshape(3,).astype(np.uint8)

    # Ubah ke format image
    img = rgb_uint8.reshape(1, 1, 3)

    # Konversi ke LAB OpenCV
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB).astype(np.float64)

    # Konversi ke CIE standar
    l = lab[..., 0] * (100.0 / 255.0)
    a = lab[..., 1] - 128.0
    b = lab[..., 2] - 128.0

    # Normalisasi ke 0–1
    l_norm = l / 100.0
    a_norm = (a + 128.0) / 255.0
    b_norm = (b + 128.0) / 255.0

    lab_norm = np.stack([l_norm, a_norm, b_norm], axis=-1)

    return lab_norm.reshape(3,)

Konversi sRGB ke HSV Ternomalisasi

In [16]:
def rgb_to_hsv_normalized(rgb_uint8: np.ndarray) -> np.ndarray:
    """
    Mengonversi satu piksel RGB (uint8) ke ruang warna HSV
    menggunakan OpenCV, lalu menormalkan hasilnya ke rentang 0–1.

    Proses konversi:
    1) RGB (uint8, 0–255)
    2) Konversi ke HSV menggunakan OpenCV (cv2.COLOR_RGB2HSV)
    3) Normalisasi:
       - H dibagi 179 (karena OpenCV menggunakan rentang 0–179)
       - S dibagi 255
       - V dibagi 255

    Parameter
    ----------
    rgb_uint8 : np.ndarray
        Array RGB berukuran (3,) dengan tipe uint8
        dalam rentang 0–255.

    Returns
    -------
    np.ndarray
        Array HSV ter-normalisasi berukuran (3,)
        dengan rentang nilai:
        - H_norm ∈ [0, 1]
        - S_norm ∈ [0, 1]
        - V_norm ∈ [0, 1]
    """

    rgb_uint8 = np.asarray(rgb_uint8).reshape(3,).astype(np.uint8)

    # Ubah ke format image
    img = rgb_uint8.reshape(1, 1, 3)

    # Konversi ke HSV OpenCV
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV).astype(np.float64)

    # Normalisasi ke 0–1
    h_norm = hsv[..., 0] / 179.0
    s_norm = hsv[..., 1] / 255.0
    v_norm = hsv[..., 2] / 255.0

    hsv_norm = np.stack([h_norm, s_norm, v_norm], axis=-1)

    return hsv_norm.reshape(3,)

Konversi sRGB ke YUV Ternomalisasi

In [17]:
def rgb_to_yuv_normalized(rgb_uint8: np.ndarray) -> np.ndarray:
    """
    Mengonversi satu piksel RGB (uint8) ke ruang warna YUV
    menggunakan OpenCV, lalu menormalkan hasilnya ke rentang 0–1.

    Proses konversi:
    1) RGB (uint8, 0–255)
    2) Konversi ke YUV menggunakan OpenCV (cv2.COLOR_RGB2YUV)
    3) Penyesuaian channel:
       - Y tetap dalam skala 0–255
       - U dan V digeser dengan -128 untuk mendapatkan nilai signed
    4) Normalisasi:
       - Y dibagi 255
       - U dan V diskalakan kembali ke 0–1

    Parameter
    ----------
    rgb_uint8 : np.ndarray
        Array RGB berukuran (3,) dengan tipe uint8
        dalam rentang 0–255.

    Returns
    -------
    np.ndarray
        Array YUV ter-normalisasi berukuran (3,)
        dengan rentang nilai:
        - Y_norm ∈ [0, 1]
        - U_norm ∈ [0, 1]
        - V_norm ∈ [0, 1]
    """

    rgb_uint8 = np.asarray(rgb_uint8).reshape(3,).astype(np.uint8)

    # Ubah ke format image
    img = rgb_uint8.reshape(1, 1, 3)

    # Konversi ke YUV OpenCV
    yuv = cv2.cvtColor(img, cv2.COLOR_RGB2YUV).astype(np.float64)

    # Pisahkan channel
    y = yuv[..., 0]
    u = yuv[..., 1] - 128.0
    v = yuv[..., 2] - 128.0

    # Normalisasi ke 0–1
    y_norm = y / 255.0
    u_norm = (u + 128.0) / 255.0
    v_norm = (v + 128.0) / 255.0

    yuv_norm = np.stack([y_norm, u_norm, v_norm], axis=-1)

    return yuv_norm.reshape(3,)

Konversi sRGB ke CMYK Ternomalisasi

In [18]:
def rgb_to_cmyk(rgb01: np.ndarray) -> np.ndarray:
    """
    Mengonversi satu piksel RGB (rentang 0–1) ke ruang warna CMYK.

    Proses konversi:
    1) Pastikan nilai RGB berada dalam rentang 0–1.
    2) Hitung K (black key):
       K = 1 - max(R, G, B)
    3) Jika K = 1 (hitam penuh), maka C = M = Y = 0.
    4) Jika tidak, hitung:
       C = (1 - R - K) / (1 - K)
       M = (1 - G - K) / (1 - K)
       Y = (1 - B - K) / (1 - K)
    5) Clip hasil untuk menghindari floating point drift.

    Parameter
    ----------
    rgb01 : np.ndarray
        Array RGB berukuran (3,) dengan nilai dalam rentang 0–1.

    Returns
    -------
    np.ndarray
        Array CMYK berukuran (4,) dalam rentang 0–1:
        [C, M, Y, K]
    """

    r, g, b = np.asarray(rgb01, dtype=np.float64).reshape(3,)

    # Pastikan RGB dalam 0–1
    r = np.clip(r, 0.0, 1.0)
    g = np.clip(g, 0.0, 1.0)
    b = np.clip(b, 0.0, 1.0)

    # Hitung K (black key)
    k = 1.0 - max(r, g, b)

    # Jika hitam penuh
    if np.isclose(k, 1.0, atol=1e-8):
        return np.array([0.0, 0.0, 0.0, 1.0], dtype=np.float64)

    # Hitung C, M, Y
    denom = 1.0 - k
    c = (1.0 - r - k) / denom
    m = (1.0 - g - k) / denom
    y = (1.0 - b - k) / denom

    # Clip untuk menghindari floating drift
    cmyk = np.clip([c, m, y, k], 0.0, 1.0)

    return np.array(cmyk, dtype=np.float64)

Fungsi Ektraksi Warna Gambar

In [19]:
def extract_features_from_image(img_rgb: np.ndarray) -> dict:
    """
    Mengekstrak fitur warna berbasis rata-rata (mean color)
    dari sebuah citra RGB dan mengonversinya ke beberapa
    ruang warna ter-normalisasi (0–1).

    Fitur yang diekstrak:
    - Mean RGB (R, G, B)
    - HSV (H, S, V)
    - YUV (Y, U, V2)
    - CIE LAB (L, a, b)
    - CMYK (C, M, Y2, K)

    Parameter
    ----------
    img_rgb : np.ndarray
        Array citra RGB bertipe uint8 dengan shape (H, W, 3)
        dan nilai dalam rentang 0–255.

    Returns
    -------
    dict
        Dictionary berisi fitur warna ter-normalisasi (0–1),
        dengan key sebagai nama fitur dan value bertipe float.
    """

    # 1) Mean RGB (0..1)
    mean_rgb01 = img_rgb.astype(np.float64).mean(axis=(0, 1)) / 255.0
    mean_rgb01 = np.clip(mean_rgb01, 0.0, 1.0)

    # Untuk fungsi OpenCV
    mean_rgb_uint8 = np.round(mean_rgb01 * 255.0).astype(np.uint8)

    # 2) HSV (0..1)
    hsv = rgb_to_hsv_normalized(mean_rgb_uint8)

    # 3) YUV (0..1)
    yuv = rgb_to_yuv_normalized(mean_rgb_uint8)

    # 4) LAB (0..1)
    lab = rgb_to_lab_cie_normalized(mean_rgb_uint8)

    # 5) CMYK (0..1)
    cmyk = rgb_to_cmyk(mean_rgb01)

    # Return fitur
    features = {
        # RGB mean (0..1)
        "R": float(mean_rgb01[0]),
        "G": float(mean_rgb01[1]),
        "B": float(mean_rgb01[2]),

        # HSV
        "H": float(hsv[0]),
        "S": float(hsv[1]),
        "V": float(hsv[2]),

        # YUV
        "Y":  float(yuv[0]),
        "U":  float(yuv[1]),
        "V2": float(yuv[2]),

        # LAB
        "L": float(lab[0]),
        "a": float(lab[1]),
        "b": float(lab[2]),

        # CMYK
        "C":  float(cmyk[0]),
        "M":  float(cmyk[1]),
        "Y2": float(cmyk[2]),
        "K":  float(cmyk[3]),
    }

    return features

#### Proses Augmentasi

In [20]:
augmenter = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),

        A.Rotate(
            limit=15,
            border_mode=cv2.BORDER_REFLECT_101,
            p=0.5
        ),

        A.Affine(
            translate_percent={"x": (-0.05, 0.05), "y": (-0.05, 0.05)},
            scale=(0.90, 1.10),
            rotate=0,
            shear=0,
            mode=cv2.BORDER_REFLECT_101,
            p=0.5
        ),

        A.RandomResizedCrop(
            size=(224, 224),
            scale=(0.8, 1.0),
            ratio=(0.9, 1.1),
            p=0.5
        ),
    ]
)

/tmp/ipython-input-560356286.py:12: UserWarning: Argument(s) 'mode' are not valid for transform Affine
  A.Affine(


Fungsi mengambil nama folder

In [21]:
def list_hour_folders(root_dir: str):
    """
    Mengambil daftar folder dalam direktori tertentu yang
    bernama angka (digit), lalu mengurutkannya secara numerik
    dari yang terkecil.

    Parameter
    ----------
    root_dir : str
        Path direktori utama yang berisi folder-folder jam

    Returns
    -------
    list
        List berisi nama folder (str) yang hanya terdiri dari digit,
        diurutkan berdasarkan nilai numeriknya dari kecil ke besar.
    """

    hours = sorted(
        [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d)) and d.isdigit()],
        key=lambda x: int(x)
    )
    return hours

Fungsi mengambil alamat tiap gambar dalam folder

In [22]:
def list_images_in_folder(folder: str):
    """
    Mengambil daftar file gambar dalam suatu folder
    berdasarkan ekstensi yang ditentukan pada variabel global `IMG_EXTS`.

    Parameter
    ----------
    folder : str
        Path folder yang akan dipindai.

    Returns
    -------
    list
        List berisi path lengkap file gambar yang ditemukan,
        diurutkan secara alfabetik.
    """

    files_ = []
    for fn in os.listdir(folder):
        if fn.lower().endswith(IMG_EXTS):
            files_.append(os.path.join(folder, fn))
    return sorted(files_)

#### Proses Pemrosesan Data Menjadi Dataframe

In [25]:
def build_dataframe_from_root(root_dir: str, dataset_name: str) -> pd.DataFrame:
  """
  Membangun DataFrame fitur dari struktur dataset berbasis folder jam.

  Struktur dataset diasumsikan:
  root_dir/
      ├── 1/
      ├── 2/
      ├── 3/
      └── ...
  di mana setiap folder merepresentasikan nilai jam (hour).

  Untuk setiap gambar:
  - Resize ke (224, 224)
  - Ekstraksi fitur warna (RGB, HSV, YUV, LAB, CMYK)
  - Tambahkan target klasifikasi (Kesegaran)
  - Tambahkan target regresi (Sisa_Usia_Jam)
  - Tambahkan metadata (Hour, Source_Image, Augment_ID)
  - Lakukan augmentasi sebanyak `AUG_PER_IMAGE`

  Parameter
  ----------
  root_dir : str
      Path direktori utama dataset.
  dataset_name : str
      Nama dataset (untuk logging/progress display).

  Returns
  -------
  pd.DataFrame
      DataFrame berisi seluruh fitur warna, target klasifikasi,
      target regresi, dan metadata gambar.

  Side Effects
  ------------
  - Membaca file gambar menggunakan OpenCV.
  - Melakukan resize gambar ke 224x224.
  - Melakukan augmentasi menggunakan variabel global `augmenter`.
  - Menampilkan progress proses ke console.
  """

  rows = []
  hour_folders = list_hour_folders(root_dir)

  # Konversi ke integer
  hour_values = sorted([int(h) for h in hour_folders])

  print(f"Memproses {dataset_name} | total folder jam: {len(hour_folders)}")

  for hour_str in hour_folders:
      hour = int(hour_str)
      hour_path = os.path.join(root_dir, hour_str)

      img_paths = list_images_in_folder(hour_path)

      # Target klasifikasi
      kesegaran = freshness_label_from_hour(hour)

      # Target regresi
      sisa_usia = max(hour_values) - hour

      for img_path in img_paths:
        # Membaca Gambar
        bgr = cv2.imread(img_path)

        # Konversi BGR ke RGB
        img_rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

        # Resize gambar
        img_rgb = cv2.resize(img_rgb, (224, 224), interpolation=cv2.INTER_AREA)

        # Inisiasi Feature Awal
        features = extract_features_from_image(img_rgb)
        features["Kesegaran"] = kesegaran
        features["Sisa_Usia_Jam"] = sisa_usia
        features["Hour"] = hour
        features["Source_Image"] = os.path.basename(img_path)
        features["Augment_ID"] = 0
        rows.append(features)

        # Proses Augmentasi
        for k in range(1, AUG_PER_IMAGE + 1):
            augmented = augmenter(image=img_rgb)["image"]
            features_aug = extract_features_from_image(augmented)
            features_aug["Kesegaran"] = kesegaran
            features_aug["Sisa_Usia_Jam"] = sisa_usia
            features_aug["Hour"] = hour
            features_aug["Source_Image"] = os.path.basename(img_path)
            features_aug["Augment_ID"] = k
            rows.append(features_aug)

  df = pd.DataFrame(rows)

  print(f"Selesai {dataset_name} | shape = {df.shape}")
  return df

## Menjalankan Proses

In [26]:
# Setting augmentasi
AUG_PER_IMAGE = 3
IMG_EXTS = (".jpg", ".jpeg", ".png")

df_train = build_dataframe_from_root(TRAIN_DIR, "TRAIN")
df_test  = build_dataframe_from_root(TEST_DIR,  "TEST")

# Preview Data
display(df_train.head())
print("\nLabel Kesegaran (TRAIN):")
print(df_train["Kesegaran"].value_counts())

Memproses TRAIN | total folder jam: 11
Selesai TRAIN | shape = (440, 21)
Memproses TEST | total folder jam: 11
Selesai TEST | shape = (440, 21)


,R,G,B,H,S,V,Y,U,V2,L,...,b,C,M,Y2,K,Kesegaran,Sisa_Usia_Jam,Hour,Source_Image,Augment_ID
0,0.615560,0.149837,0.086466,0.022346,0.858824,0.615686,0.282353,0.403922,0.796078,0.352941,...,0.654902,0.0,0.756584,0.859533,0.384440,Segar,10,0,Screenshot 2025-12-27 113308.png,0
1,0.588167,0.143237,0.082623,0.022346,0.858824,0.588235,0.270588,0.407843,0.780392,0.337255,...,0.650980,0.0,0.756469,0.859525,0.411833,Segar,10,0,Screenshot 2025-12-27 113308.png,1
2,0.531592,0.129412,0.074676,0.022346,0.858824,0.533333,0.243137,0.419608,0.756863,0.301961,...,0.635294,0.0,0.756557,0.859523,0.468408,Segar,10,0,Screenshot 2025-12-27 113308.png,2
3,0.616087,0.149930,0.086197,0.022346,0.858824,0.615686,0.282353,0.403922,0.796078,0.352941,...,0.654902,0.0,0.756642,0.860090,0.383913,Segar,10,0,Screenshot 2025-12-27 113308.png,3
4,0.638303,0.159199,0.093406,0.022346,0.850980,0.639216,0.298039,0.400000,0.800000,0.368627,...,0.654902,0.0,0.750590,0.853665,0.361697,Segar,10,0,Screenshot 2025-12-27 113354.png,0



Label Kesegaran (TRAIN):
Kesegaran
Segar           200
Tidak Segar     120
Kurang Segar    120
Name: count, dtype: int64


## Mengunduk Dataframe

In [27]:
OUTDIR = "/content/output_csv"
os.makedirs(OUTDIR, exist_ok=True)

train_csv_path = os.path.join(OUTDIR, "Dataset_Raw_Train.csv")
test_csv_path  = os.path.join(OUTDIR, "Dataset_Raw_Test.csv")

# Simpan tanpa index
df_train.to_csv(train_csv_path, index=False)
df_test.to_csv(test_csv_path, index=False)

print("CSV tersimpan:")
print(train_csv_path)
print(test_csv_path)

# Download ke lokal
files.download(train_csv_path)
files.download(test_csv_path)

CSV tersimpan:
/content/output_csv/Dataset_Raw_Train.csv
/content/output_csv/Dataset_Raw_Test.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>